In [ ]:
from keras.models import Sequential,Model
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D, ZeroPadding3D
from tensorflow.keras.optimizers import SGD
from keras.layers import Add,Input,Conv3D,AveragePooling3D

In [ ]:
# single C3D model

def C3D_model(summary=False):
    """ Return the Keras model of the network
    """
    model = Sequential()
    # 1st layer group
    # input are 16 frames with size 112*112 with 3 channels
    model.add(Convolution3D(64, (3,3,3), activation='relu', 
                            padding='same', name='conv1',
                            strides=(1, 1, 1), 
                            input_shape=(16, 112, 112,3)))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), 
                           padding='valid', name='pool1'))
    # 2nd layer group
    model.add(Convolution3D(128, (3,3,3), activation='relu', 
                            padding='same', name='conv2',
                            strides=(1, 1, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), 
                           padding='valid', name='pool2'))
    # 3rd layer group
    model.add(Convolution3D(256, (3,3,3), activation='relu', 
                            padding='same', name='conv3a',
                            strides=(1, 1, 1)))
    model.add(Convolution3D(256, (3,3,3), activation='relu', 
                            padding='same', name='conv3b',
                            strides=(1, 1, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), 
                           padding='valid', name='pool3'))
    # 4th layer group
    model.add(Convolution3D(512, (3,3,3), activation='relu', 
                            padding='same', name='conv4a',
                            strides=(1, 1, 1)))
    model.add(Convolution3D(512, (3,3,3), activation='relu', 
                            padding='same', name='conv4b',
                            strides=(1, 1, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), 
                           padding='valid', name='pool4'))
    # 5th layer group
    model.add(Convolution3D(512, (3,3,3), activation='relu', 
                            padding='same', name='conv5a',
                            strides=(1, 1, 1)))
    model.add(Convolution3D(512, (3,3,3), activation='relu', 
                            padding='same', name='conv5b',
                            strides=(1, 1, 1)))
    model.add(ZeroPadding3D(padding=(0, 1, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), 
                           padding='valid', name='pool5'))
    model.add(Flatten())
    # FC layers group
    model.add(Dense(4096, activation='relu', name='fc6'))
    model.add(Dropout(.5))
    model.add(Dense(4096, activation='relu', name='fc7'))
    model.add(Dropout(.5))
    model.add(Dense(487, activation='softmax', name='fc8'))
    if summary:
        print(model.summary())
    return model

In [ ]:
#double C3D model  

def C3D_model(input,idx):
    """ Return the Keras model of the network
    """
    # 1st layer group
    # input are 16 frames with size 112*112 with 3 channels
    h1 = Conv3D(64,(3,3,3),activation='relu',padding='same', name='conv1'+idx,strides=(1, 1, 1))(input)
    p1 = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), padding='valid', name='pool1'+idx)(h1)
    # 2nd layer group
    h2 = Conv3D(128,(3,3,3), activation='relu', padding='same', name='conv2'+idx,strides=(1, 1, 1))(p1)
    p2 = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='valid', name='pool2'+idx)(h2)
    # 3rd layer group
    h3 = Conv3D(256, (3,3,3), activation='relu',padding='same', name='conv3a'+idx,strides=(1, 1, 1))(p2)
    h3_ = Conv3D(256, (3,3,3), activation='relu', padding='same', name='conv3b'+idx,strides=(1, 1, 1))(h3)
    p3 = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='valid', name='pool3'+idx)(h3_)
    # 4th layer group
    h4 = Conv3D(512, (3,3,3), activation='relu', padding='same', name='conv4a'+idx,strides=(1, 1, 1))(p3)
    h4_ = Conv3D(512, (3,3,3), activation='relu', padding='same', name='conv4b'+idx,strides=(1, 1, 1))(h4)
    p4 = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='valid', name='pool4'+idx)(h4_)
    # 5th layer group
    h5 = Conv3D(512, (3,3,3), activation='relu', padding='same', name='conv5a'+idx,strides=(1, 1, 1))(p4)
    h5_ = Conv3D(512, (3,3,3), activation='relu', padding='same', name='conv5b'+idx,strides=(1, 1, 1))(h5)
    z5 = ZeroPadding3D(padding=(0, 1, 1))(h5_)
    p5 = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='valid', name='pool5'+idx)(z5)
    f5 = Flatten()(p5)
    return f5

def double_c3d_model():
    # input 
    input = Input(shape =(16, 224, 224,3)) 
    # downsample by 2
    i1 = AveragePooling3D(pool_size=(1, 2, 2), strides=(1,2,2))(input)
    # double c3d model
    o1 = C3D_model(i1,'f')
    o2 = C3D_model(i1,'s')
    # c3d model fused
    fusion_result = Add()([o1,o2])
    # FC layers group
    d6 = Dense(4096, activation='relu', name='fc6')(fusion_result)
    dd6 = Dropout(.5)(d6)
    d7 = Dense(4096, activation='relu', name='fc7')(dd6)
    dd7 = Dropout(.5)(d7)
    output = Dense(487, activation='softmax', name='fc8')(dd7)
    # output of double C3D model
    model = Model(inputs=input,outputs=output)
    return model

model = double_c3d_model()
model.summary()
model.compile(optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["acc"])

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 16, 224, 22  0           []                               
                                4, 3)]                                                            
                                                                                                  
 average_pooling3d (AveragePool  (None, 16, 112, 112  0          ['input_1[0][0]']                
 ing3D)                         , 3)                                                              
                                                                                                  
 conv1f (Conv3D)                (None, 16, 112, 112  5248        ['average_pooling3d[0][0]']      
                                , 64)                                                         

In [ ]:
# mount on google drive
import sys
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# read the preprocessed data using pickle
import pickle
d = pickle.load(open('/content/drive/MyDrive/coms6998/d1.pkl','rb'))
val = list(d.values())

In [ ]:
import numpy as np

In [ ]:
new_val = np.asarray(val[:1800])
test_val = np.asarray(val[1800:2000])
print(new_val.shape)

(1800, 16, 224, 224, 3)


In [ ]:
#load the data as dataframe
import pandas as pd
f = pd.read_excel('/content/drive/MyDrive/coms6998/labels.xlsx')
label = list(f['label'])[:1800]
test_label = list(f['label'])[1800:2000]
print(len(label))

1800


In [ ]:
label = np.asarray(label)

In [ ]:
# train_batch generator that yield training data to model
def train_batch_generator(new_val,label):
    for i in range(3):
        print('Reading file {0}...........'.format(i))
        X_train = new_val[i*600:(i+1)*600]
        Y_train = label[i*600:(i+1)*600]
        yield (X_train, Y_train)

In [ ]:
# training the data on 1800 videos
for x_train, y_train in train_batch_generator(new_val,label):
    model.fit(x_train, y_train, batch_size = 8, epochs = 15)


Reading file 0...........
Epoch 1/15
75/75 [==============================] - 67s 745ms/step - loss: 168.8796 - acc: 0.1950
Epoch 2/15
75/75 [==============================] - 56s 745ms/step - loss: 1.8549 - acc: 0.1917
Epoch 3/15
75/75 [==============================] - 56s 745ms/step - loss: 1.8468 - acc: 0.2067
Epoch 4/15
75/75 [==============================] - 56s 745ms/step - loss: 1.8467 - acc: 0.1933
Epoch 5/15
75/75 [==============================] - 56s 744ms/step - loss: 1.8369 - acc: 0.2050
Epoch 6/15
75/75 [==============================] - 56s 744ms/step - loss: 2.1652 - acc: 0.2000
Epoch 7/15
75/75 [==============================] - 56s 745ms/step - loss: 1.8301 - acc: 0.1883
Epoch 8/15
75/75 [==============================] - 56s 745ms/step - loss: 1.8299 - acc: 0.2017
Epoch 9/15
75/75 [==============================] - 56s 745ms/step - loss: 1.8364 - acc: 0.2083
Epoch 10/15
75/75 [==============================] - 56s 744ms/step - loss: 1.8218 - acc: 0.1767
Epoch 11/15

In [ ]:
#test data on 200 videos
results = model.evaluate(np.array(test_val), np.array(test_label))

7/7 [==============================] - 6s 594ms/step - loss: 1.7163 - acc: 0.2450
